In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import spinmob as sm
import mcphysics as mphys
from scipy.optimize import curve_fit

In [25]:
#Adds together 1 hour of calibration data for a given source
def Sum(source):
    yData = [0]*2048
    i=1
    j=0
    while i <= 12:
        while j < 10:
            yData += mphys.data.load_chn('Calibration/' + source + '/' + str(i) + '/' + source + '/' + source + 
                                         ' 00' + str(j) + '.Chn')[1]
            j += 1
        i += 1
    return yData

#Gaussian, where m=peak, o=standard deviation, A=amplitude, B=Linear Coeff., C=Linear Offset
def Gaussian(x, m, o, A, B, C):
    return A*np.exp(-0.5*(((x-m)/o)**2)) + B*x + C

#Function for linear curve-fit of peaks for each channel (i.e. for final callibration curve)
def Linear(x, a, b):
    return a*x+b

#Curve fits a Gaussian to a given emitter's callibration data curve. The interval determines the width of data
#which the Gaussian fits around. 
def Fit(input_databox, interval, guess):
    xData = np.arange(0, 2048)
    param, cov = curve_fit(Gaussian, xData[guess-interval:guess+interval], 
                           input_databox[guess-interval:guess+interval], p0=[guess, 10, 10, 10, 10])
    error = np.sqrt(np.diag(cov))
    print(param), print(error)
    return [param, error]

def Chi(ydata, yFit):
    return ((ydata-yFit)**2)/(ydata**2)

Cs = Fit(Sum('Cs-137'), 300, 1742)
Ba = Fit(Sum('Ba-133'), 300, 850)
Co = Fit(Sum('Co-57'), 300, 375)
Na = Fit(Sum('Na-22'), 300, 1360)

yvalues = [662.1, 302.85, 122, 511.0]
xvalues = [Cs[0][0], Ba[0][0], Co[0][0], Na[0][0]]
errvalues = [Cs[1][0], Ba[1][0], Co[1][0], Na[1][0]]

linearFit_par, linearFit_cov = curve_fit(Linear, xvalues, yvalues, sigma = errvalues)

plt.plot(np.arange(0, 2048), Linear(np.arange(0, 2048), *linearFit_par))
plt.scatter(xvalues, yvalues)
print(Cs[0][1])
print(linearFit_par), print(np.sqrt(np.diag(linearFit_cov)))

[ 1.76481351e+03  5.75377047e+01  7.67732553e+01 -1.41975253e-02
  2.94022184e+01]
[5.12537951e-01 5.74604184e-01 6.02446800e-01 1.16206404e-03
 2.01423351e+00]
[8.25412764e+02 2.05884694e+01 3.94520218e+01 7.13101264e-02
 3.04470033e+01]
[ 7.86752497  8.14786592 13.17882783  0.01519703 13.31141471]
[ 3.57295177e+02  1.76155277e+01  1.49414650e+04 -1.84630705e+00
  1.30045024e+03]
[  0.19099991   0.19665546 141.59862258   0.15101154  63.72125323]
[ 1.37712732e+03  4.89389886e+01  1.67947261e+02 -2.16615652e-02
  4.41815620e+01]
[3.38231206e-01 3.70758284e-01 1.02323823e+00 1.80894187e-03
 2.46827787e+00]
57.53770470594125
[  0.38255271 -14.77557581]
[0.00096385 0.86160325]


(None, None)